In [1]:
import sys
import pandas as pd
import altair as alt

In [2]:
success_table=pd.read_csv("fig6_input.txt", sep='\t')
RUN_TYPES = ['filtered_exomiser_default', 'noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL']
success_table=success_table.fillna(-20)


In [ ]:
mapping = pd.read_csv('GS_ID_mapping.csv')
mapped= []
for i, row in success_table.iterrows():
    ID = row['ID']
    mapped_id = mapping[mapping['ID']==ID]['Dumb_ID'].item()
    mapped.append(mapped_id)
success_table['Dumb_ID'] = mapped

In [3]:
def default_vs_fit(unfiltered_run_type, filtered_run_type, success_table, limit,domain):
    data = []
    to_plot = []
    big_change=[]
    plot3=[]
    for i, patient_row in success_table.iterrows():
        ID = patient_row['Dumb_ID']
        unfiltered_rank = patient_row['Variant_Level_noMOI_rank_' + unfiltered_run_type]
        filtered_rank = patient_row['Variant_Level_noMOI_rank_' + filtered_run_type]
        if filtered_rank == 'N/A':
            filtered_rank = 0
        difference = abs(unfiltered_rank - filtered_rank)
       
        if unfiltered_rank > limit and filtered_rank <= limit and filtered_rank >0:
           
            to_plot.append(ID)
                # if difference >30:
                #     to_plot.append(ID)
        if unfiltered_rank <0:
            if unfiltered_rank != filtered_rank:
                if filtered_rank <= limit and filtered_rank > 0:
                    to_plot.append(ID)

        # if unfiltered_rank < filtered_rank:
        #     to_plot.append(ID)

        data.append([ID, unfiltered_rank, difference, str(unfiltered_run_type)])
        data.append([ID, filtered_rank, difference, str(filtered_run_type)])

    df = pd.DataFrame(data, columns=['ID', 'Rank', 'Diff', 'RunType'])
    print(len(to_plot) + len(big_change))


    source = df[df['ID'].isin(to_plot)].sort_values(by=['Rank'], ascending=False)
    order = list(source['ID'])
    print(len(source)/2)
    rank_changes = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',opacity=1, strokeWidth=1).encode(
        x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)),
        y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
        color=alt.Color('RunType',  scale=alt.Scale(scheme='set1', domain=domain), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ['RunType', 'ID', 'Rank', 'Diff']
    ).properties(
        height=250,
        width=150
    )

    source = df[df['ID'].isin(plot3)].sort_values(by='Diff', ascending=False)
    order = list(source['ID'])
    print(len(source)/2)
    

    return df, to_plot, big_change, rank_changes



In [4]:
data = []
to_plot = []
domain = ['','filtered_exomiser_default','noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL']

unfiltered_run_type = 'filtered_exomiser_default'
filtered_run_type = 'noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL'
df, to_plot, big_change,rank_changes_gs_exomiser = default_vs_fit(unfiltered_run_type,filtered_run_type,success_table,  10, domain)


69
70.0
0.0


In [5]:
rank_changes_gs_exomiser.properties(height=300,width=800)


alt.Chart(...)